In [23]:
import os
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd

# setting up authorization
cid ="915b0386b44c43348183c89a679f7e0a" 
secret = "71089ac558bd4bea8db1ceeadffe735a"

os.environ['SPOTIPY_CLIENT_ID']= cid
os.environ['SPOTIPY_CLIENT_SECRET']= secret
os.environ['SPOTIPY_REDIRECT_URI']='http://localhost:8888/callback'

username = "chinmayaandukuri"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope)

In [28]:
if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_top_tracks(limit=50,offset=0,time_range='short_term')
    results = results['items']
else:
    print("Can't get token for", username)

In [61]:
track_names = []
track_artists = []
track_album_names = []
track_cover_art_urls = []
largest_images = []
track_ids = []
for track_dict in results:
    track_names.append(track_dict['name'])
    track_artists.append(track_dict['artists'][0]['name'])
    track_album_names.append(track_dict['album']['name'])
    track_cover_art_urls.append(track_dict['album']['images'])
    largest_images.append(track_dict['album']['images'][0]['url'])
    track_ids.append(track_dict['id'])

In [62]:
# Each song's cover art urls entry is a list of 3 dictionaries
# Access this list of 3 dictionaries for song i with track_cover_art_urls[i]
# track_cover_art_urls[i][0] is 640 x 640
# track_cover_art_urls[i][1] is 300 x 300
# track_cover_art_urls[i][2] is 64 x 64
print(track_cover_art_urls[0])

[{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273c6e0948bbb0681ff29cdbae8', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e02c6e0948bbb0681ff29cdbae8', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d00004851c6e0948bbb0681ff29cdbae8', 'width': 64}]


In [74]:
genres = []
for artist in track_artists:
    result = sp.search(artist, limit=1, type="artist")
    artist_genres = result["artists"]["items"][0]["genres"]
    # Note that for each artist, we get a LIST of genres associated
    # Eventually we'll calculate genre by getting the UNION of all of the artist's genres
    # For now, this works. We might make it more complex in the future
    # genres should be a list of 50 lists if we have 50 artists.
    genres.append(artist_genres)

In [75]:
df = pd.DataFrame(list(zip(track_names, track_artists, track_album_names, track_ids, track_cover_art_urls, largest_images, genres)),
               columns =['Name', 'Artist', 'Album', 'ID', 'Cover Art Information', 'Cover Art Largest Image Link', 'Genres'])
df['index'] = pd.Series(list(range(1, len(track_names) + 1)))
df.set_index(df['index'], inplace = True)

In [83]:
### Top 5 songs sub-dataframe
top5 = df.head(5)[['Name', 'Artist', 'Cover Art Largest Image Link']]
top5

,Name,Artist,Cover Art Largest Image Link
index,,,
1,January 28th,J. Cole,https://i.scdn.co/image/ab67616d0000b273c6e094...
2,Too Many Nights (feat. Don Toliver & with Future),Metro Boomin,https://i.scdn.co/image/ab67616d0000b27313e54d...
3,family ties (with Kendrick Lamar),Baby Keem,https://i.scdn.co/image/ab67616d0000b2731bfa23...
4,On Time (with John Legend),Metro Boomin,https://i.scdn.co/image/ab67616d0000b27313e54d...
5,Alone With You,Alina Baraz,https://i.scdn.co/image/ab67616d0000b273947c33...


In [76]:
### Top artist calculation here
top_artist = df['Artist'].mode()
print(top_artist)

0    Kendrick Lamar
dtype: object


In [78]:
### Top genre calculation
genres_union = []
# Find the union of genres - rap may occur 100 times, hip hop 40, etc.
for genre_list in list(df['Genres']):
    genres_union.extend(genre_list)

# This calculation finds the genre that occurs the most number of times
top_genre = max(set(genres_union), key=genres_union.count)
top_genre_formatted = top_genre.title()